In [1]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.least_confidence import LeastConfidence
from al_methods.entropy_sampling import EntropySampling
from al_methods.batch_BALD import BatchBALD
from al_methods.core_set import CoreSet
from ssl_methods.semi_fixmatch import fixmatch
from ssl_methods.semi_flexmatch import flexmatch
from ssl_methods.semi_pseudolabel import pseudolabel
import models
from torchvision import transforms
from framework.framework1 import Framework1
import framework
import torch.nn as nn
import time
from torch.utils.data import DataLoader
from scipy.spatial.distance import jensenshannon
import al_methods
import os
import seaborn as sns 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
sns.set_theme()

/home/nasserali/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Configuration

In [2]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },

            'svhn':
                {
                 'n_class':10,
                'channels':3,
                'size': 32,
                'transform_tr': transforms.Compose([ 
                                    transforms.RandomCrop(size = 32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.4377, 0.4438, 0.4728), 'std': (0.1980, 0.2010, 0.1970)},
                },
            'cifar10':
                {
                 'n_class':10,
                 'channels':3,
                 'size': 32,
                 'transform_tr': transforms.Compose([
                                    transforms.RandomCrop(size = 32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]),
                 'loader_tr_args':{'batch_size': 256, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 512, 'num_workers': 8},
                 'normalize':{'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2470, 0.2435, 0.2616)},
                 },


            'cifar100': 
               {
                'n_class':100,
                'channels':3,
                'size': 32,
                'transform_tr': transforms.Compose([
                                transforms.RandomCrop(size = 32, padding=4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))]),
                'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))]),
                'loader_tr_args':{'batch_size': 2048, 'num_workers': 4},
                'loader_te_args':{'batch_size': 512, 'num_workers': 8},
                'normalize':{'mean': (0.5071, 0.4867, 0.4408), 'std': (0.2675, 0.2565, 0.2761)},
                }
        }

### Dataset

In [3]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [4]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()

In [5]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=6 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=1 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print("begin with : ",NUM_INIT_LB," Number of round",NUM_ROUND," How many to query",NUM_QUERY)


begin with :  600  Number of round 5  How many to query 600


In [6]:
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file 
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

array([False, False, False, ..., False, False, False])

### Models

In [7]:
model=models.__dict__["ResNet50"](n_class=args_pool['mnist']['n_class'])

In [8]:
model.feature_extractor.conv1=torch.nn.Conv2d(args_pool['mnist']['channels'],16,kernel_size=3,stride=1,padding=1,bias=False)
model.discriminator.dis_fc2=torch.nn.Linear(in_features=50,out_features=args_pool['mnist']['n_class'],bias=True)

### Test methods 

In [9]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.framework='framwork1'
        self.optimizer='SGD'
        self.save_model=False
        self.ALstrat='LeastConfidence'
        self.SSLstrat='fixmatch'
        self.n_epoch=10

In [10]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [11]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

In [12]:
framework_1= Framework1(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [13]:
framework_1.train(alpha=2e-3,n_epoch=5)

Let's use 1 GPUs!
[Batch=000] [Loss=2.46]

==>>[2023-08-29 08:27:06] [Epoch=000/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=2.70]

==>>[2023-08-29 08:27:15] [Epoch=001/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:45] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.24]

==>>[2023-08-29 08:27:24] [Epoch=002/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:30] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=1.99]

==>>[2023-08-29 08:27:32] [Epoch=003/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:19] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=1.57]

==>>[2023-08-29 08:27:41] [Epoch=004/005] [framwork1(LeastConfidence+fixmatch) Need: 00:00:09] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
---- save figure the accuracy/loss curve of train/val into ./save/mnist


0.1688

### Data Augmentation

In [14]:
# Create a weakly data augmentation transform
weak_transform = transforms.Compose([
    # transforms.RandomCrop(14),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

# Create a strongly data augmentation transform
strong_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 0.1)),
    transforms.RandomPerspective(),
    transforms.ToTensor()
])


### Apply on data

#### Unlabeled data

In [15]:
# Find indices where the boolean matrix is False
false_indices = np.argwhere(framework_1.idxs_lb == False).squeeze()

U_X_tr=X_tr[false_indices]
U_Y_tr=Y_tr[false_indices]

unlabeled_data_weak=DataLoader(handler(U_X_tr,U_Y_tr,transform=weak_transform),batch_size=64)
unlabeled_data_strong=DataLoader(handler(U_X_tr,U_Y_tr,transform=strong_transform),batch_size=64)

### Train weakly

In [33]:
model_weak=framework_1.model.eval()
predictions_weak = []
y_true=[]
with torch.no_grad():
    for x, y,_ in unlabeled_data_weak:
        # Move the batch to the appropriate device (CPU or GPU)
        x= x.to(framework_1.device)  # device could be 'cuda' or 'cpu'
        y= y.to(framework_1.device)
        y_true.append(y.cpu().numpy())
        # Forward pass to obtain predictions
        batch_predictions,_= model_weak(x)
        
        # Append batch predictions to the list
        predictions_weak.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Concatenate predictions from all batches
predictions_weak = np.concatenate(predictions_weak, axis=0)


### Train strongly

In [28]:
#Strong aug train
model_strong=framework_1.model.eval()
predictions_strong = []
with torch.no_grad():
    for x, _,_ in unlabeled_data_strong:
        # Move the batch to the appropriate device (CPU or GPU)
        x= x.to(framework_1.device)  # device could be 'cuda' or 'cpu'
        
        # Forward pass to obtain predictions
        batch_predictions,_ = model_strong(x)
        
        # Append batch predictions to the list
        predictions_strong.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Concatenate predictions from all batches
predictions_strong = np.concatenate(predictions_strong, axis=0)


In [29]:
predictions_strong

array([[ 0.6336532 , 18.292686  ,  8.283445  , ..., 14.054209  ,
         5.446602  ,  9.29574   ],
       [ 0.62113285, 18.12209   ,  8.214174  , ..., 13.9443035 ,
         5.3892994 ,  9.214708  ],
       [ 0.63561743, 18.30967   ,  8.291466  , ..., 14.066608  ,
         5.4533997 ,  9.303816  ],
       ...,
       [ 0.6276991 , 18.195902  ,  8.242558  , ..., 13.99092   ,
         5.4155965 ,  9.250857  ],
       [ 0.6313265 , 18.240788  ,  8.261713  , ..., 14.021614  ,
         5.430856  ,  9.27487   ],
       [ 0.6363763 , 18.323977  ,  8.297624  , ..., 14.075201  ,
         5.457615  ,  9.309856  ]], dtype=float32)

In [30]:
np.argmax(predictions_strong,axis=1)

array([1, 1, 1, ..., 1, 1, 1])

#### True predictions

In [37]:
m=np.argmax(predictions_strong,axis=1)-np.concatenate(y_true,axis=0)

In [42]:
m[3]

0

In [43]:
np.where(m==0)[0].shape

(6683,)

In [34]:
# np.save('predictions_strong.np',predictions_strong)
# np.save('predictions_weak.npy',predictions_weak)

In [16]:
predictions_weak=np.load('predictions_weak.npy')
predictions_strong=np.load('predictions_strong.np.npy')

#### labaled data

In [17]:
true_indices = np.argwhere(framework_1.idxs_lb == True).squeeze()
L_X_tr=X_tr[true_indices]
L_Y_tr=Y_tr[true_indices]

labeled_data_weak=DataLoader(handler(L_X_tr,L_Y_tr,transform=weak_transform),batch_size=32)
labeled_data_strong=DataLoader(handler(L_X_tr,L_Y_tr,transform=strong_transform),batch_size=32)

In [18]:
L_X_tr.shape

(600, 28, 28)

In [19]:
model_weak=framework_1.model.eval()
predictions_weak_l = []
with torch.no_grad():
    for x, _,_ in labeled_data_weak:
        # Move the batch to the appropriate device (CPU or GPU)
        x= x.to(framework_1.device)  # device could be 'cuda' or 'cpu'
        print(x.shape)
        
        # Forward pass to obtain predictions
        batch_predictions,_= model_weak(x)
        
        # Append batch predictions to the list
        predictions_weak_l.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Concatenate predictions from all batches
predictions_weak_l = np.concatenate(predictions_weak_l, axis=0)


torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
torch.Size([32, 1, 28, 28])
x:  torch.Size([32, 1, 28, 28])
out:  torch.

In [20]:
#Strong aug train
model_strong=framework_1.model.eval()
predictions_strong_l = []
with torch.no_grad():
    for x, _,_ in labeled_data_strong:
        # Move the batch to the appropriate device (CPU or GPU)
        x= x.to(framework_1.device)  # device could be 'cuda' or 'cpu'
        
        # Forward pass to obtain predictions
        batch_predictions,_ = model_strong(x)
        
        # Append batch predictions to the list
        predictions_strong_l.append(batch_predictions.cpu().numpy())  # Move predictions to CPU and convert to numpy

# Concatenate predictions from all batches
predictions_strong_l = np.concatenate(predictions_strong_l, axis=0)


x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])
out:  torch.Size([32, 512])
emb:  torch.Size([32, 512])
x:  torch.Size([32, 1, 28, 28])


### Stability

#### unlabeled data

In [21]:
jensen=jensenshannon(predictions_weak,predictions_strong,axis=1)
# np.max(predictions_weak,axis=1)
# 0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))
tot=(1-jensen ) *0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))

/home/nasserali/anaconda3/lib/python3.10/site-packages/scipy/spatial/distance.py:1300: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


In [22]:
jensen.shape

(59400,)

In [32]:
plt.figure(figsize=(10,8))
sns.histplot(tot,kde=True)
plt.show()

#### labeled data

In [33]:
jensen_l=jensenshannon(predictions_weak_l,predictions_strong_l,axis=1)
# np.max(predictions_weak,axis=1)
# 0.5*(np.max(predictions_weak,axis=1)+np.max(predictions_strong,axis=1))

/home/nasserali/anaconda3/lib/python3.10/site-packages/scipy/spatial/distance.py:1300: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(js / 2.0)


In [34]:
jensen_l.shape

(600,)

In [26]:
np.max(predictions_weak_l,axis=1).shape

(600,)

In [27]:
np.max(predictions_strong_l,axis=1).shape

(600,)

In [35]:
tot_l=(1-jensen_l ) *0.5*(np.max(predictions_weak_l,axis=1)+np.max(predictions_strong_l,axis=1))

In [36]:
plt.figure(figsize=(10,8))
sns.histplot(tot_l,kde=True)
plt.show()

### put togeter

In [30]:
# Create a figure and array of subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 8))  # Change the number of subplots as needed

In [31]:
# Plot the histograms using sns.histplot
sns.histplot(tot, ax=axes[0], kde=True, color='blue', label='Data 1')
axes[0].set_title('Histogram of Data 1')
axes[0].legend()

sns.histplot(tot_l, ax=axes[1], kde=True, color='orange', label='Data 2')
axes[1].set_title('Histogram of Data 2')
axes[1].legend()

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [37]:
# Create a figure and array of subplots
fig, ax = plt.subplots(figsize=(10, 8))  # Change the number of subplots as needed
# Plot the histograms using sns.histplot
sns.histplot(tot, ax=ax, kde=True, color='blue', label='Data 1')
sns.histplot(tot_l, ax=ax, kde=True, color='orange', label='Data 2')


# Set title, labels, and legend
ax.set_title('Superposed Histograms')
ax.set_xlabel('X-axis Label')
ax.set_ylabel('Frequency')
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()

### Combination

In [ ]:
print(f' Sratgey for active learning{args.ALstrat} and strategy for semi-supervised learning used {args.SSLstrat}')
stratAl=LeastConfidence(framework_1.X_tr, framework_1.Y_tr, framework_1.X_te, framework_1.Y_te, framework_1.idxs_lb, framework_1.net, framework_1.handler, framework_1.args,framework_1.n_pool,framework_1.device)
stratSSL=pseudolabel(framework_1.X_tr, framework_1.Y_tr, framework_1.X_te, framework_1.Y_te, framework_1.idxs_lb, framework_1.net, framework_1.handler, framework_1.args,framework_1.n_pool,framework_1.device,framework_1.predict,framework_1.g)


test_acc=framework_1.predict(framework_1.X_te,framework_1.Y_te)
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

for rd in range(1, NUM_ROUND+1):
    
    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(framework_1.n_pool)[framework_1.idxs_lb])
        if NUM_QUERY > int(nEnd*framework_1.n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*framework_1.n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = stratAl.query(NUM_QUERY)
        q_idxs = output
        framework_1.idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_1.update(framework_1.idxs_lb)
        if hasattr(stratAl, 'train'):
        
            best_test_acc=stratAl.train(alpha=2e-3, n_epoch=10)
        else: best_test_acc = framework_1.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(framework_1.n_pool)[framework_1.idxs_lb])
        if NUM_QUERY > int(nEnd*framework_1.n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*framework_1.n_pool/100) - labeled
            
        # query
        ts = time.time()

        output = stratSSL.query(NUM_QUERY)
        q_idxs = output
        framework_1.idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_1.update(framework_1.idxs_lb)
        best_test_acc = stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc

In [43]:
import numpy as np

# Example data
X_tr_t = np.array([1, 2, 3, 4, 5])
Y_tr_t = np.array([10, 20, 30, 40, 50])
false_indices_t = np.array([False, True, False, True, False])

# # Check data types and boolean values
# false_indices = bool(false_indices)

# Filter using the boolean mask
L_X_tr_t = X_tr_t[~false_indices_t]
L_Y_tr_t = Y_tr_t[~false_indices_t]

print(L_X_tr_t)  # Output: [2, 4]
print(L_Y_tr_t)  # Output: [20, 40

[1 3 5]
[10 30 50]
